# Parameter-Efficient Fine-Tuning with LoRA

In this notebook, we will:
- Load a pre-trained GPT-2 model
- Evaluate its performance on a sequence classification task
- Perform parameter-efficient fine-tuning using LoRA
- Compare the performance of the fine-tuned model to the original model

In [ ]:
%pip install transformers datasets peft evaluate scikit-learn

## Load Pre-trained Model and Tokenizer

In [ ]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from datasets import load_dataset

# Load pre-trained model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.config.pad_token_id = model.config.eos_token_id

## Load Dataset

We'll use the `imdb` dataset from the Hugging Face datasets library, which is suitable for binary sequence classification.

In [2]:
# Load dataset
dataset = load_dataset("imdb")

# Preprocess the dataset
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)

## Evaluate the Original Model

We'll use the Hugging Face `Trainer` to evaluate the model.

In [ ]:
from transformers import Trainer, TrainingArguments
import numpy as np
import evaluate

# Define evaluation metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_eval_batch_size=8,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

# Evaluate the model
original_results = trainer.evaluate()

## Perform Parameter-Efficient Fine-Tuning with LoRA

We'll use the `peft` library to apply LoRA to the GPT-2 model.

In [ ]:
from peft import LoraConfig, get_peft_model

# Create a PEFT config
peft_config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.1, 
    target_modules=["c_attn"],
    task_type="SEQ_CLS"
)

# Create a PEFT model
peft_model = get_peft_model(model, peft_config)

## Train the PEFT Model

We'll fine-tune the model using the same `Trainer` setup.

In [ ]:
# Update Trainer for PEFT model
trainer.model = peft_model
trainer.train_dataset=encoded_dataset["train"]
# Train the model
trainer.train()

## Save the Trained Model

In [8]:
# Save the fine-tuned model
peft_model.save_pretrained("./peft_model")

## Evaluate the Fine-Tuned Model

Finally, we'll evaluate the fine-tuned model and compare its performance to the original model.

In [ ]:
# Evaluate the fine-tuned model
fine_tuned_results = trainer.evaluate()

# Compare results
print("Original Model Accuracy:", original_results['eval_accuracy'])
print("Fine-Tuned Model Accuracy:", fine_tuned_results['eval_accuracy'])